1. Nuget Packages

In [ ]:
#r "nuget:Microsoft.ML,*-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;

using System.IO;
using System.Text;

2. Data Models

In [ ]:
public class Penguin
{
    [LoadColumn(0)]
    public bool Label {get;set;}

    [LoadColumn(2)]
    public float CulmenLength {get;set;}

    [LoadColumn(3)]
    public float CulmenDepth {get;set;}
}

public class Predictions
{
    public bool PredictedLabel{get;set;}
}

3. Load and Split Data

In [ ]:
var mlContext = new MLContext();
var trainingDataView = mlContext.Data.LoadFromTextFile<Penguin>("../../ML.Net.Training/Datasets/penguins_size_binary.csv", hasHeader: true, separatorChar: ',');
var dataSplit = mlContext.Data.TrainTestSplit(trainingDataView, testFraction: 0.3);

In [ ]:
dataSplit.TestSet.ToTabularDataResource().Display();

4. Train the Model

In [ ]:
var model = mlContext.BinaryClassification.Trainers.LbfgsLogisticRegression(labelColumnName: "Label", featureColumnName: "Features");

var pipeline = mlContext.Transforms.Concatenate("Features", 
                nameof(Penguin.CulmenLength),
                nameof(Penguin.CulmenDepth))
                .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features")
                .Append(model));

var trainedModel = pipeline.Fit(dataSplit.TrainSet);

5. Evaluate Model

In [ ]:
var testSetTransform = trainedModel.Transform(dataSplit.TestSet);
var metrics = mlContext.BinaryClassification.Evaluate(testSetTransform);

In [ ]:
metrics.Accuracy

6. Save Model

In [ ]:
//ML.Net models are saved in the .mdl extension
mlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, "./BinaryClassificationModel.mdl");

7. Load Model and Predict

In [ ]:
var newSample = new Penguin
{
    CulmenDepth = 18.4f,
    CulmenLength = 40.8f
};

using (var stream = new FileStream("./BinaryClassificationModel.mdl", FileMode.Open, FileAccess.Read, FileShare.Read))
{
    var loadedModel = mlContext.Model.Load(stream, out _);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<Penguin, Predictions>(loadedModel);
    var results = predictionEngine.Predict(newSample);

    results.Display();
}